In [1]:
#!/usr/bin/env python
# coding: utf-8

import gc
import time 
import scipy.io 
import numpy as np
import pandas as pd
import seaborn as sns 
import tensorflow as tf
import tqdm as notebook_tqdm
import matplotlib.pyplot as plt
from numpy import ravel 
from sklearn.preprocessing import OneHotEncoder,KBinsDiscretizer,LabelEncoder,OrdinalEncoder
from sklearn.preprocessing import normalize
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import confusion_matrix,classification_report
from tensorflow.keras import layers, models
from keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.metrics import (
    ## Classification metrics
    accuracy_score,				# Accuracy classification score.
    auc,						# Compute Area Under the Curve (AUC) using the trapezoidal rule.
    average_precision_score,	# Compute average precision (AP) from prediction scores.
    balanced_accuracy_score,	# Compute the balanced accuracy.
    brier_score_loss,			# Compute the Brier score loss.
    #class_likelihood_ratios,	# Compute binary classification positive and negative likelihood ratios.
    classification_report,		# Build a text report showing the main classification metrics.
    cohen_kappa_score,			# Compute Cohen's kappa: a statistic that measures inter-annotator agreement.
    confusion_matrix,			# Compute confusion matrix to evaluate the accuracy of a classification.
    dcg_score,					# Compute Discounted Cumulative Gain.
    det_curve,					# Compute error rates for different probability thresholds.
    f1_score,					# Compute the F1 score, also known as balanced F-score or F-measure.
    fbeta_score,				# Compute the F-beta score.
    hamming_loss,				# Compute the average Hamming loss.
    hinge_loss,					# Average hinge loss (non-regularized).
    jaccard_score,				# Jaccard similarity coefficient score.
    log_loss,					# Log loss, aka logistic loss or cross-entropy loss.
    matthews_corrcoef,			# Compute the Matthews correlation coefficient (MCC).
    multilabel_confusion_matrix,	# Compute a confusion matrix for each class or sample.
    ndcg_score,					# Compute Normalized Discounted Cumulative Gain.
    precision_recall_curve,		# Compute precision-recall pairs for different probability thresholds.
    precision_recall_fscore_support,	# Compute precision, recall, F-measure and support for each class.
    precision_score,			# Compute the precision.
    recall_score,				# Compute the recall.
    roc_auc_score,				# Compute Area Under the Receiver Operating Characteristic Curve (ROC AUC)     from prediction scores.
    roc_curve,					# Compute Receiver operating characteristic (ROC).
    top_k_accuracy_score,		# Top-k Accuracy classification score.
    zero_one_loss,				# Zero-one classification loss.

    ## Regression metrics
#     explained_variance_score,	# Explained variance regression score function.
#     max_error,					# The max_error metric calculates the maximum residual error.
#     mean_absolute_error,		# Mean absolute error regression loss.
#     mean_squared_error,			# Mean squared error regression loss.
#     mean_squared_log_error,		# Mean squared logarithmic error regression loss.
#     median_absolute_error,		# Median absolute error regression loss.
#     mean_absolute_percentage_error,	# Mean absolute percentage error (MAPE) regression loss.
#     r2_score,					# coefficient of determination or regression score function.
#     mean_poisson_deviance,		# Mean Poisson deviance regression loss.
#     mean_gamma_deviance,		# Mean Gamma deviance regression loss.
#     mean_tweedie_deviance,		# Mean Tweedie deviance regression loss.
#     d2_tweedie_score,			# D^2 regression score function, fraction of Tweedie deviance explained.
#     mean_pinball_loss,			# Pinball loss for quantile regression.
#     d2_pinball_score,			# regression score function, fraction of pinball loss explained.
#     d2_absolute_error_score,	# regression score function, fraction of absolute error explained.
)

import warnings
warnings.filterwarnings("ignore")
ignore_warnings = True

pd.set_option("display.precision", 4)
pd.set_option("display.float_format", lambda x: "%.4f" % x)


In [3]:

# Perform Prediction:
levels = ["grams","genus","serotypes","strains",]
grams_class = {0: '+', 1: '-'}
genus_class = {0: 'Bacillus', 1: 'Cronobacter', 2: 'Enterobacter', 3: 'Escherichia', 4: 'Listeria', 5: 'Salmonella', 6: 'Shigella', 7: 'Staphylococcus', 8: 'Vibrio'}
species_class = {
		0: 'Bacillus_cereus', 1: 'Cronobacter_dublin', 2: 'Cronobacter_malonaticus', 3: 'Cronobacter_sakazakii', 4: 'Cronobacter_turicensis', 5: 'Enterobacter_hormaechei', 6: 'Escherichia_coli', 7: 'Listeria_grayi', 8: 'Listeria_innocua', 9: 'Listeria_monocytogenes', 
		10: 'Listeria_welshimeri', 11: 'Salmonella_aberdeen', 12: 'Salmonella_choleraesuis', 13: 'Salmonella_derby', 14: 'Salmonella_enteritidis', 15: 'Salmonella_heidelberg', 16: 'Salmonella_london', 17: 'Salmonella_paratyphi_A', 18: 'Salmonella_typhimurium', 19: 'Salmonella_typhimurium_monophasic_variant', 
		20: 'Shigella_boydii', 21: 'Shigella_flexneri', 22: 'Staphylococcus_aureus', 23: 'Vibrio_parahaemolyticus'
			}
strains_class = {
    0: 'MRL-040001', 1: 'MRL-040002', 2: 'MRL-040003', 3: 'MRL-040004', 4: 'MRL-040005', 5: 'MRL-040006', 6: 'MRL-040007', 7: 'MRL-040008', 8: 'MRL-040009', 9: 'MRL-040010', 
	10: 'MRL-040011', 11: 'MRL-040204', 12: 'MRL-040205', 13: 'MRL-040233', 14: 'MRL-040237', 15: 'MRL-040239', 16: 'MRL-040248', 17: 'MRL-040259', 18: 'MRL-0419078', 19: 'MRL-300125', 
	20: 'MRL-300126', 21: 'MRL-300127', 22: 'MRL-300128', 23: 'MRL-300129', 24: 'MRL-300130', 25: 'MRL-300131', 26: 'MRL-300132', 27: 'MRL-300133', 28: 'MRL-300134', 29: 'MRL-300140', 
	30: 'MRL-300141', 31: 'MRL-300142', 32: 'MRL-300143', 33: 'MRL-300144', 34: 'MRL-300145', 35: 'MRL-300146', 36: 'MRL-300147', 37: 'MRL-300148', 38: 'MRL-300149', 39: 'MRL-AP4001', 
	40: 'MRL-AP4002', 41: 'MRL-AP4003', 42: 'MRL-AP4004', 43: 'MRL-AP4005', 44: 'MRL-AP4006', 45: 'MRL-AP4007', 46: 'MRL-AP4008', 47: 'MRL-AP4009', 48: 'MRL-AP4010', 49: 'MRL-AP4011', 
	50: 'MRL-AP4012', 51: 'MRL-AP4013', 52: 'MRL-AP4014', 53: 'MRL-AP4015', 54: 'MRL-AP4016', 55: 'MRL-AP4017', 56: 'MRL-AP4018', 57: 'MRL-AP4019', 58: 'MRL-AP4020', 59: 'MRL-AP4021', 
	60: 'MRL-AP4022', 61: 'MRL-AP4023', 62: 'MRL-AP4024', 63: 'MRL-AP4025', 64: 'MRL-AP4026', 65: 'MRL-AP4027', 66: 'MRL-AP4028', 67: 'MRL-AP4029', 68: 'MRL-AP4030', 69: 'MRL-AP4031', 
	70: 'MRL-AP4032', 71: 'MRL-AP4033', 72: 'MRL-AP4034', 73: 'MRL-AP4035', 74: 'MRL-AP4036', 75: 'MRL-AP4037', 76: 'MRL-AP4038', 77: 'MRL-AP4039', 78: 'MRL-AP4040', 79: 'MRL-AP4041', 
	80: 'MRL-AP4042', 81: 'MRL-AP4043', 82: 'MRL-AP4044', 83: 'MRL-AP4045', 84: 'MRL-AP4046', 85: 'MRL-AP4047', 86: 'MRL-AP4048', 87: 'MRL-AP4049', 88: 'MRL-AP4050', 89: 'MRL-AP4051', 
	90: 'MRL-AP4052', 91: 'MRL-AP4053', 92: 'MRL-AP4054', 93: 'MRL-AP4055', 94: 'MRL-AP4056', 95: 'MRL-AP4057', 96: 'MRL-AP4058', 97: 'MRL-AP4059', 98: 'MRL-AP4060', 99: 'MRL-AP4061', 
	100: 'MRL-AP4062', 101: 'MRL-AP4063', 102: 'MRL-AP4064', 103: 'MRL-AP4065', 104: 'MRL-AP4066', 105: 'MRL-AP4067', 106: 'MRL-AP4068', 107: 'MRL-AP4069', 108: 'MRL-AP4070', 109: 'MRL-AP5001', 
	110: 'MRL-AP5002', 111: 'MRL-AP5003', 112: 'MRL-AP5004', 113: 'MRL-AP5005', 114: 'MRL-AP5006', 115: 'MRL-AP5007', 116: 'MRL-AP5008', 117: 'MRL-AP5009', 118: 'MRL-AP5010', 119: 'MRL-AP5011', 
	120: 'MRL-AP5012', 121: 'MRL-AP5013', 122: 'MRL-AP5014', 123: 'MRL-AP5015', 124: 'MRL-AP5016', 125: 'MRL-AP5017', 126: 'MRL-AP5018', 127: 'MRL-AP5019', 128: 'MRL-AP5020', 129: 'MRL-AP5021', 
	130: 'MRL-AP5022', 131: 'MRL-AP5023', 132: 'MRL-AP5024', 133: 'MRL-AP5025', 134: 'MRL-AP5026', 135: 'MRL-AP5027', 136: 'MRL-AP5028', 137: 'MRL-AP5029', 138: 'MRL-AP5030', 139: 'MRL-AP5031', 
	140: 'MRL-AP5200', 141: 'MRL-AP5201', 142: 'MRL-AP5202', 143: 'MRL-AP5203', 144: 'MRL-AP5204', 145: 'MRL-AP5205', 146: 'MRL-AP5206', 147: 'MRL-AP5207', 148: 'MRL-AP5208', 149: 'MRL-AP5209', 
	150: 'MRL-AP5210', 151: 'MRL-AP5232', 152: 'MRL-AP5234', 153: 'MRL-AP5235', 154: 'MRL-AP5236', 155: 'MRL-AP5238', 156: 'MRL-AP5240', 157: 'MRL-AP5251', 158: 'MRL-AP5252', 159: 'MRL-AP5261', 
	160: 'MRL-AP5262', 161: 'MRL-AP5263', 162: 'MRL-AP5265', 163: 'MRL-AP5266', 164: 'MRL-AP5267', 165: 'MRL-AP5268', 166: 'MRL-AP5269', 167: 'MRL-BC0001', 168: 'MRL-BC0002', 169: 'MRL-BC0003', 
	170: 'MRL-BC0004', 171: 'MRL-BC0204', 172: 'MRL-BC0212', 173: 'MRL-BC0231', 174: 'MRL-BC0241', 175: 'MRL-BC0250', 176: 'STD-ATCC12022', 177: 'STD-ATCC12868', 178: 'STD-ATCC13076', 179: 'STD-ATCC14028', 
	180: 'STD-ATCC17802', 181: 'STD-ATCC19112', 182: 'STD-ATCC19114', 183: 'STD-ATCC19115', 184: 'STD-ATCC25922', 185: 'STD-ATCC27660', 186: 'STD-ATCC29213', 187: 'STD-ATCC29544', 188: 'STD-ATCC33847', 189: 'STD-ATCC43251', 
	190: 'STD-ATCC43889', 191: 'STD-ATCC43895', 192: 'STD-ATCC9207', 193: 'STD-CICC10297', 194: 'STD-CICC21493', 195: 'STD-CICC21670', 196: 'STD-CICC21672', 197: 'STD-CMCC50093', 198: 'STD-IQCC10511'
		}

Accuracy = []
B_Accuracy = []
F1 = []
Kappa = []
MCC = []
Names = []
Precision = []
Recall = []
ROC_AUC = []
Times = []

def make_scores(jobname,y_test,y_pred):
    # summarize the fit of the model
    start = time.time()
    localtime = time.strftime("%Y%m%d%H%M%S", time.localtime())
    print(("Predicting Accuracy: %.4f%%") % (accuracy_score(y_test,y_pred,normalize=True)*100.0))
    print(classification_report(y_test,y_pred))
    
    ConfusionMat1 = confusion_matrix(y_test,y_pred)
    
    df_cm = pd.DataFrame(ConfusionMat1)
    df_cm.to_csv("ConfusionMatrix_"+jobname+"_"+localtime+"-1.csv")
    
    ConfusionMat2 = np.array(ConfusionMat1)
    ConfusionMat2 = 100.0*ConfusionMat2.astype('float') / ConfusionMat2.sum(axis=1)[:, np.newaxis]
    print(ConfusionMat2)

    df_cm = pd.DataFrame(ConfusionMat2)
    df_cm.to_csv("ConfusionMatrix_"+jobname+"_"+localtime+"-2.csv")

    accuracy = accuracy_score(y_test, y_pred, normalize=True)
    b_accuracy = balanced_accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test,y_pred,average="weighted")
    precision = precision_score(y_test,y_pred,average="weighted")
    recall = recall_score(y_test,y_pred,average="weighted")
    mcc = matthews_corrcoef(y_test,y_pred)
    kappa = cohen_kappa_score(y_test,y_pred)

    try:
        roc_auc = roc_auc_score(y_test, y_pred)
    except Exception as exception:
        roc_auc = None
        if ignore_warnings is False:
            print("ROC AUC couldn't be calculated for " + name)
            print("Convert your string labels to numeric labels explicitly instead.")
            print(exception)
    Names.append(jobname)
    Accuracy.append(accuracy)
    B_Accuracy.append(b_accuracy)
    ROC_AUC.append(roc_auc)
    F1.append(f1)
    Kappa.append(kappa)
    Precision.append(precision)
    Recall.append(recall)
    MCC.append(mcc)
    Times.append(time.time() - start)
    return Names,Accuracy,B_Accuracy,ROC_AUC,F1,Kappa,Precision,Recall,MCC,Times

def predict(jobname,pkl_path,x_test,y_test=None):
    tested_results = pd.DataFrame([])
    y_test = np.argmax(y_test, axis=1)

    pkl_path = Path(pkl_path)
    mod_pkls = list(pkl_path.glob("*.h5"))
    
    i = 0
    for pkl_file in mod_pkls:
        print("-"*100)
        i += 1
        modelfile = pkl_path/pkl_file
        print(modelfile)
        model=tf.keras.models.load_model(modelfile)
        
        start = time.time()
        y_pred0 = model.predict(x_test)
        print(y_pred0)
        if i == 1: 
            y_prob = y_pred0
        else:
            y_prob += y_pred0
        y_pred = np.argmax(y_pred0, axis=1)
        tested_results[i] = np.argmax(y_pred0, axis=1)
        
        del model
        gc.collect()

        if y_test is not None: 
            Names,Accuracy,B_Accuracy,ROC_AUC,F1,Kappa,Precision,Recall,MCC,Times = make_scores(str(i),y_test,y_pred)
        #break
    
    print("#"*100)
    print("Perform reinforcement prediction ...")
    start = time.time()
    tested_results["pred0"] = np.argmax(y_prob/float(i), axis=1)
    tested_results["pred1"] = [tested_results.loc[i,:].value_counts().idxmax() for i in range(tested_results.shape[0])]
    tested_results["pred2"] = [strains_class[tested_results.loc[i,:].value_counts().idxmax()] for i in range(tested_results.shape[0])]
    
    if y_test is not None:
        y_pred = tested_results["pred1"]
        
        Names,Accuracy,B_Accuracy,ROC_AUC,F1,Kappa,Precision,Recall,MCC,Times = make_scores(jobname,y_test,y_pred)

        scores = pd.DataFrame({
                                    "Model": Names,
                                    "Accuracy": Accuracy,
                                    "Balanced_Accuracy": B_Accuracy,
                                    "Precision": Precision,
                                    "ROC_AUC": ROC_AUC,
                                    "Recall": Recall,
                                    "Kappa": Kappa,
                                    "MCC": MCC,
                                    "F1": F1,
                                    "Time": Times,
                                })
        print(scores)
    
    return tested_results,scores if y_test is not None else None

## 模型加载
from pathlib import Path

start = time.time()
jobname = "Strains"
base_path = ""

# load dataset
data1 = pd.read_csv('../../Raman_MRL+STD_data/Raman_MRL+STD_data_400to1799_preprocessed_1.csv',sep=',', header=0, names=None, index_col=None)

head(data1)

## firstly divide into groups："grams","species","genus","strains"
classname = "strains"
data2 = data1.drop(columns = [ L for L in levels if L != classname])
X2 = data2.iloc[:,:-1]
Y = data2.iloc[:, -1]

# Encode the labels into unique integers
encoder = LabelEncoder()
Y2 = encoder.fit_transform(ravel(Y))
label_code = set(zip(Y2,Y))
label_code = {k:v for (k,v) in sorted(label_code, key=lambda label_code : label_code[0])} 
#print("\nlabel_code:\n",label_code)

encoder = OneHotEncoder()
Y3 = encoder.fit_transform(Y2.reshape(-1, 1)).toarray()
print(Y3)

# label_code2 = set(zip(Y3,Y))
# label_code2 = {k:v for (k,v) in sorted(label_code2, key=lambda label_code2 : label_code2[0])} 
# print("\nlabel_code:\n",label_code2)

X = np.array(X2)
Y = np.array(Y)
Y_CNN = Y3
n_class = 199

print('Shape of Input Data =', X.shape)
print('Shape of Label Y_CNN =', Y_CNN.shape)
print('Shape of Label Y =', Y.shape)
print('Number of Classification n_class =', n_class)

gc.collect()
print("\nDataset is Loaded.\n")
print("="*100)
print("\nStart training...")

kSplits = 15
kfold = KFold(n_splits=kSplits, random_state=32, shuffle=True)
Input_1D = X.reshape([-1,467,1])
X_1D_train, X_1D_test, y_1D_train, y_1D_test = train_test_split(Input_1D, Y_CNN, train_size=0.8,test_size=0.2, random_state=101)

del data1
del data2
del X
del X2
del Input_1D
del X_1D_train
gc.collect()

print(X_1D_test)
print(y_1D_test)
#print('Shape of Input Data =', X.shape)
print('Shape of Input Data =', X_1D_test.shape)
print('Shape of Input Data =', y_1D_test.shape)

tested_results,scores = predict(jobname,base_path,X_1D_test,y_1D_test)
print(tested_results)


       400.66  403.661  406.661  409.662  412.663  415.664  418.664  421.665  \
0     -1.1184  -1.5097  -1.4495  -1.2375  -1.1171  -1.0534  -1.1550  -1.5582   
1     -1.3117  -0.6583  -1.2642  -0.9805  -1.3879  -1.5824  -1.4282  -2.0688   
2     -1.1831  -1.4530  -1.1467  -0.7359  -1.1102  -1.5109  -1.4224  -1.0650   
3     -1.1950  -1.2410  -0.9328  -0.8096  -1.1231  -1.3436  -1.1575  -0.8066   
4     -1.4714  -1.3323  -1.3686  -1.4431  -1.4082  -1.1231  -1.0258  -2.0546   
...       ...      ...      ...      ...      ...      ...      ...      ...   
46714 -0.7139  -1.2588  -1.2409  -1.1490  -1.0385  -1.1671  -1.4753  -1.8650   
46715 -0.3447  -1.1440  -1.2279  -0.9813  -0.8854  -1.1612  -1.3963  -1.6847   
46716 -1.3663  -0.8130  -1.0411  -1.0031  -1.0351  -1.4765  -1.7507  -1.9158   
46717 -0.2900  -0.4842  -0.8618  -1.0746  -1.2155  -1.5290  -1.7985  -2.0270   
46718 -0.2951  -0.8779  -0.9482  -0.9679  -1.0759  -1.3433  -1.5376  -1.7562   

       424.666  427.667  ...  1784.01  

----------------------------------------------------------------------------------------------------
Raman-keras-tf-1D-CNN-MRL+STD_data1_Strains_model-loops10-0.9849-2.h5
292/292 [==============================] - 3s 8ms/step
[[2.65191497e-35 2.61622454e-35 1.45027359e-33 ... 5.86203130e-25
  4.67388885e-24 1.00000000e+00]
 [2.57423013e-27 3.33846359e-27 2.37077314e-23 ... 3.34767376e-18
  4.90191442e-31 3.98297490e-33]
 [3.50228991e-32 9.79326384e-33 1.03949445e-30 ... 1.73319439e-35
  2.53043380e-22 1.06181051e-21]
 ...
 [6.39638168e-29 1.95143037e-25 4.33123239e-20 ... 3.14634834e-18
  8.73946890e-17 1.00000000e+00]
 [5.01976312e-33 0.00000000e+00 4.23659816e-36 ... 2.82526646e-34
  9.58692498e-35 2.05489033e-30]
 [6.48287306e-22 5.28055394e-17 9.26317898e-23 ... 1.43546762e-18
  1.00000000e+00 4.94997066e-09]]
Predicting Accuracy: 98.4910%
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        38
           1       1.00      1.00  

292/292 [==============================] - 2s 7ms/step
[[2.1173070e-30 9.9945718e-26 1.0851289e-31 ... 7.0888490e-22
  4.8945807e-27 1.0000000e+00]
 [3.0610272e-17 7.0262230e-16 1.8733591e-19 ... 3.3321842e-16
  2.2004949e-20 5.9728095e-20]
 [1.3384709e-28 6.7555867e-30 1.0245014e-34 ... 2.2077832e-29
  8.7730168e-22 4.5500628e-29]
 ...
 [3.2737495e-30 1.3631228e-26 8.9003705e-29 ... 1.6452824e-25
  4.9172165e-23 1.0000000e+00]
 [4.4651658e-32 8.3933382e-36 1.3043243e-32 ... 5.7829971e-28
  1.8536807e-28 5.7525895e-28]
 [8.0030160e-23 1.2341196e-11 5.5817382e-25 ... 2.0389550e-21
  1.0000000e+00 1.5368718e-14]]
Predicting Accuracy: 98.6836%
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        38
           1       0.95      1.00      0.98        41
           2       1.00      0.95      0.98        44
           3       0.97      0.97      0.97        34
           4       1.00      1.00      1.00        39
           5       1.00   

----------------------------------------------------------------------------------------------------
Raman-keras-tf-1D-CNN-MRL+STD_data1_Strains_model-loops12-0.9853-2.h5
292/292 [==============================] - 2s 7ms/step
[[3.0549970e-24 3.3580509e-23 2.0295634e-23 ... 2.4114596e-18
  1.1486270e-18 1.0000000e+00]
 [1.3504893e-18 2.0872777e-19 1.9505883e-26 ... 1.4146075e-21
  8.4730983e-26 5.3561060e-24]
 [4.4770992e-27 2.7556556e-25 8.3216717e-28 ... 5.0740039e-30
  5.0793530e-21 6.2474452e-22]
 ...
 [4.5504909e-24 8.9472084e-22 2.6529923e-24 ... 6.3955847e-19
  1.7252097e-16 1.0000000e+00]
 [3.3659459e-26 5.8226927e-26 4.8809561e-25 ... 4.7756641e-24
  2.2137574e-26 6.9735941e-23]
 [1.2550577e-20 1.1512392e-14 2.2952696e-19 ... 3.7153333e-17
  9.9999988e-01 6.2758545e-08]]
Predicting Accuracy: 98.5338%
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        38
           1       0.98      1.00      0.99        41
           2     

----------------------------------------------------------------------------------------------------
Raman-keras-tf-1D-CNN-MRL+STD_data1_Strains_model-loops13-0.9841-2.h5
292/292 [==============================] - 3s 7ms/step
[[9.30131628e-30 7.00286635e-30 4.24892039e-26 ... 8.03842810e-20
  1.34492206e-17 1.00000000e+00]
 [7.05611467e-32 9.89203323e-21 1.28550382e-25 ... 1.55375432e-22
  1.00928039e-18 7.45105934e-26]
 [3.07633660e-18 2.81704274e-20 1.11145731e-20 ... 1.66765943e-15
  1.20989898e-14 1.07261602e-19]
 ...
 [6.21074120e-27 1.36191431e-30 1.99671557e-26 ... 1.75694454e-14
  4.06128608e-17 1.00000000e+00]
 [2.07329515e-34 5.18918824e-35 1.41891234e-32 ... 1.06398291e-30
  7.33639049e-31 7.00401241e-31]
 [7.82912249e-29 4.70448299e-14 7.99107238e-27 ... 9.04451153e-26
  1.00000000e+00 1.04009154e-11]]
Predicting Accuracy: 98.4161%
              precision    recall  f1-score   support

           0       1.00      0.97      0.99        38
           1       0.98      1.00  

...

----------------------------------------------------------------------------------------------------
Raman-keras-tf-1D-CNN-MRL+STD_data1_Strains_model-loops7-0.9824-2.h5
292/292 [==============================] - 3s 8ms/step
[[8.52255018e-37 1.25159070e-29 2.52645085e-29 ... 2.14071774e-33
  7.50605256e-21 1.00000000e+00]
 [0.00000000e+00 5.78433473e-38 1.37031531e-37 ... 3.02496964e-34
  0.00000000e+00 9.54375354e-36]
 [1.09419167e-29 3.93766922e-25 6.35790343e-27 ... 8.06234314e-37
  1.47199789e-28 1.35492184e-18]
 ...
 [8.02085875e-32 5.64339343e-27 3.45846173e-26 ... 1.54345085e-23
  4.92478974e-20 1.00000000e+00]
 [1.57331355e-34 2.59236769e-38 3.29201564e-33 ... 2.28988922e-36
  1.86220647e-36 1.22747365e-26]
 [3.07675012e-29 9.12184405e-17 1.11619379e-27 ... 2.53500878e-23
  9.99998689e-01 1.29456930e-06]]
Predicting Accuracy: 98.2449%
              precision    recall  f1-score   support

           0       0.97      1.00      0.99        38
           1       1.00      0.95   

292/292 [==============================] - 3s 8ms/step
[[1.0092769e-25 8.3446529e-21 2.2696240e-23 ... 9.4181926e-19
  1.4613366e-17 1.0000000e+00]
 [2.4108478e-22 2.4009463e-21 3.6319610e-21 ... 3.1300564e-25
  3.1862031e-20 3.8160666e-26]
 [2.8985555e-23 7.0494921e-18 8.9218818e-23 ... 1.1002712e-18
  4.2387573e-17 9.0803382e-16]
 ...
 [1.6382097e-20 1.3735493e-15 2.6198110e-19 ... 1.9280255e-18
  1.9337296e-15 1.0000000e+00]
 [1.9142831e-27 1.1860094e-28 5.1362205e-24 ... 5.1840025e-28
  3.5234482e-25 4.2636066e-22]
 [4.7669966e-16 3.9282045e-07 6.4393886e-11 ... 3.7733870e-12
  9.9860007e-01 1.3992134e-03]]
Predicting Accuracy: 98.5766%
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        38
           1       0.95      1.00      0.98        41
           2       1.00      0.95      0.98        44
           3       1.00      0.97      0.99        34
           4       0.97      0.97      0.97        39
           5       0.96   

----------------------------------------------------------------------------------------------------
Raman-keras-tf-1D-CNN-MRL+STD_data1_Strains_model-loops9-0.9845-2.h5
292/292 [==============================] - 2s 7ms/step
[[6.1650313e-23 4.3022459e-22 2.0978086e-17 ... 7.5666783e-15
  3.9127054e-19 1.0000000e+00]
 [1.4950981e-20 8.0789412e-20 1.2030127e-15 ... 8.6785843e-21
  7.8173907e-21 4.6308185e-22]
 [3.7120573e-19 2.7301711e-22 6.3706504e-19 ... 3.1131375e-16
  3.8271616e-18 7.4550529e-15]
 ...
 [4.9638152e-19 5.3436292e-19 3.7121423e-19 ... 5.5368243e-13
  3.0116771e-15 1.0000000e+00]
 [1.0501854e-35 4.4958383e-35 2.1551606e-28 ... 4.9306179e-27
  4.7673807e-32 6.4734872e-28]
 [2.0856917e-21 1.7296775e-16 9.4674031e-18 ... 8.4423211e-18
  1.0000000e+00 2.3158167e-10]]
Predicting Accuracy: 98.4589%
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        38
           1       1.00      1.00      1.00        41
           2      

####################################################################################################
Perform reinforcement prediction ...
Predicting Accuracy: 99.4863%
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        38
           1       1.00      1.00      1.00        41
           2       1.00      1.00      1.00        44
           3       1.00      1.00      1.00        34
           4       1.00      1.00      1.00        39
           5       1.00      1.00      1.00        45
           6       1.00      1.00      1.00        36
           7       1.00      1.00      1.00        40
           8       1.00      1.00      1.00        33
           9       1.00      0.96      0.98        25
          10       0.97      1.00      0.99        35
          11       1.00      1.00      1.00        30
          12       1.00      1.00      1.00        32
          13       1.00      1.00      1.00        40
          14       1.

      Model  Accuracy  Balanced_Accuracy  Precision ROC_AUC  Recall  Kappa  \
0         1    0.9841             0.9805     0.9846    None  0.9841 0.9839   
1         2    0.9849             0.9810     0.9854    None  0.9849 0.9848   
2         3    0.9868             0.9834     0.9873    None  0.9868 0.9867   
3         4    0.9853             0.9816     0.9858    None  0.9853 0.9852   
4         5    0.9842             0.9801     0.9846    None  0.9842 0.9840   
5         6    0.9865             0.9837     0.9869    None  0.9865 0.9864   
6         7    0.9830             0.9793     0.9837    None  0.9830 0.9829   
7         8    0.9890             0.9864     0.9893    None  0.9890 0.9889   
8         9    0.9843             0.9802     0.9847    None  0.9843 0.9842   
9        10    0.9856             0.9821     0.9860    None  0.9856 0.9854   
10       11    0.9856             0.9824     0.9861    None  0.9856 0.9854   
11       12    0.9854             0.9813     0.9860    None  0.9